In [1]:
import testing_util_v2
import python_terminal_command
import pickle
import pandas as pd

In [2]:
pred_tc_finetune = pickle.load(open('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/save/APPS/codet5-testcase-v2/checkpoint-10000/prediction-5.pkl', 'rb'))
pred_tc_ppo_one = pickle.load(open('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/save/APPS/codet5-testcase-v2/checkpoint-10000/PPO_functional-1/step-65/prediction-5.pkl', 'rb'))
pred_tc_ppo_multi455 = pickle.load(open('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/save/APPS/codet5-testcase-v2/checkpoint-10000/PPO_functional-2/step-455/prediction-5.pkl', 'rb'))
pred_tc_ppo_multi887 = pickle.load(open('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/save/APPS/codet5-testcase-v2/checkpoint-10000/PPO_functional-2/step-887/prediction-5.pkl', 'rb'))
pred_tc_ppo_multi72 = pickle.load(open('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/save/APPS/codet5-testcase-v2/checkpoint-10000/PPO_functional-4/step-72/prediction-5.pkl', 'rb'))

In [3]:
df_test = pd.read_csv('/home/wannita/HDD18TB/active_repo/pyRL/text-to-testcase/dataset/APPS_new/apps_test_3765.csv')

In [4]:
pred_tc_finetune_1 = [t[0] for t in pred_tc_finetune]
pred_tc_ppo_one_1 = [t[0] for t in pred_tc_ppo_one]
pred_tc_ppo_multi455_1 = [t[0] for t in pred_tc_ppo_multi455]
pred_tc_ppo_multi887_1 = [t[0] for t in pred_tc_ppo_multi887]
pred_tc_ppo_multi72_1 = [t[0] for t in pred_tc_ppo_multi72]

In [5]:
commander = python_terminal_command.PythonTerminalCommand('coverage')

In [ ]:
pred_text = pred_tc_ppo_multi72_1
# executable
tmp_reward, error = testing_util_v2.test_function(prompts=df_test['prompt_code'],
                                solutions=df_test['output_solution'], 
                                testcases=pred_text, 
                                fn_names=df_test['fn_name'],
                                debug=False)
for i in range(len(df_test)):

    # code coverage
    if not error:
        script = testing_util_v2.transform_to_input(prompts=df_test['prompt_code'][i],
                                    solutions=df_test['output_solution'][i], 
                                    testcases=pred_text[i], 
                                    fn_names=df_test['fn_name'][i],
                                    filter_syntax=True)
        err, coverage_output, executable = commander.process_coverage_test(script)
        coverage_score = int(coverage_output.split()[-1][:-1])

In [21]:
s = '''import sys
import io
def test_call_solution(input_string):  # pragma: no cover
    sys.stdin = io.StringIO(input_string)
    captured_output = io.StringIO()
    sys.stdout = captured_output
    call_solution()
    sys.stdin = sys.__stdin__
    sys.stdout = sys.__stdout__
    actual_output = captured_output.getvalue().strip()
    return actual_output
def call_solution():
    r"""
    Anton has the integer x. He is interested what positive integer, which doesn't exceed x, has the maximum sum of digits.
    
    Your task is to help Anton and to find the integer that interests him. If there are several such integers, determine the biggest of them. 
    
    
    -----Input-----
    
    The first line contains the positive integer x (1 ≤ x ≤ 10^18) — the integer which Anton has. 
    
    
    -----Output-----
    
    Print the positive integer which doesn't exceed x and has the maximum sum of digits. If there are several such integers, print the biggest of them. Printed integer must not contain leading zeros.
    
    
    -----Examples-----
    Input
    100
    
    Output
    99
    
    Input
    48
    
    Output
    48
    
    Input
    521
    
    Output
    499
    """
    num = list(map(int, input()))
    best = num[:]
    for i in range(-1, -len(num) - 1, -1):
        if num[i] == 0:
            continue
        num[i] -= 1
        for j in range(i + 1, 0):
            num[j] = 9
        if sum(num) > sum(best):
            best = num[:]
    s = ''.join(map(str, best)).lstrip('0')
    print(s)

# check the correctness of the `call_solution` function
assert test_call_solution('99') == "99"'''

In [22]:
err, output, executable = commander.process_coverage_test(s)

In [23]:
print('error:', err)
print('executable:', executable)
print('output:', output)

error: 
executable: True
output: Name              Stmts   Miss  Cover

-------------------------------------

testcoverage.py      16      2    88%

-------------------------------------

TOTAL                16      2    88%



In [24]:
int(output.split()[-1][:-1])

88